# NIRSpec IFU Data Reduction Pipeline

By the JWST ERS TEMPLATES team (B. Welch, +). 
Early draft version, Sept. 2022. 

Based on a combination of notebooks from STSCI:  
https://github.com/STScI-MIRI/MRS-ExampleNB/blob/main/Flight_Notebook1/MRS_FlightNB1.ipynb
https://github.com/spacetelescope/jwebbinar_prep/blob/main/spec_mode/spec_mode_stage_2.ipynb
https://github.com/spacetelescope/jwebbinar_prep/blob/main/spec_mode/spec_mode_stage_3.ipynb
https://github.com/spacetelescope/jwebbinar_prep/blob/main/ifu_session/jwebbinar5_nirspecifu.ipynb

Prerequisites: Install JWST pipeline. See TEMPLATES pipeline installation notebook (https://github.com/JWST-Templates/Notebooks/blob/main/0_install_pipeline.ipynb) for help.

# Example Pipeline Run on NIRSpec IFU Data
Currently this is the re-observed IFU data on SDSS-1723.
There is a lot of work still to do before we fully understand the best practices for running this pipeline. For a while this notebook will contain a lot of trial and error. It may be a bit ugly, but hopefully it is useful!

### Note!
This version is applying background subtraction in the Stage 3 pipeline. It takes the Stage 2 output extracted 1-D spectra ("x1d.fits") from the background observations, and creates an average spectrum to universally subtract from the science data. 

There is another possibility for background subtraction, where the dedicated background exposures would be subtracted from the science exposures during the Stage 2 pipeline. These would be "image from image" subtractions, rather than creating a universal background spectrum. I'm planning to work up a version of this kind of background subtraction soon. 

It's not clear which method is going to perform the best, so we're finding out! 


#### ---- note from TAH:  
the installation notebook in this repo specifies a different CDRS_PATH definition, which is fine although perhaps for the skae of the users of these notebooks we should make that path definition the same as used in this notebook? or vice-versa

In [1]:
import numpy as np
import glob
import os

# Modify the paths to the relevant directories on your machine
# ------------------------------------------------------------
# 1) point to where the jwst pipeline config files are located
#home = "/Users/bdwelch1/Documents/" # for B. Welch
home = "/Users/tahutch1/programs/jwst-drp/" # for T. Hutchison

# 2) point to where you keep your data
# input_path = '/Users/bdwelch1/Documents/data/templates/sdss1723/retake_filegrab_take2/JWST/' # for B. Welch
input_path = "/Users/tahutch1/data/raw/jwst/ers/templates/MAST_2022-09-23T1410/JWST/" # for T. Hutchison

# 3) point to where you want your processed outputs to live
# output_path = '/Users/bdwelch1/Documents/data/templates/sdss1723/pipeline_testrun_take2/' # for B. Welch
output_path = "/Users/tahutch1/data/raw/jwst/ers/templates/reduced/" # for T. Hutchison


###############################################
# IMPORTANT!! Make sure to use 'jwst-crds-pub'! 
# This will contain the most up to date reference files
###############################################
os.environ["CRDS_PATH"] = home + "crds_cache/jwst_pub"
os.environ["CRDS_SERVER_URL"] = "https://jwst-crds-pub.stsci.edu"

import zipfile
import urllib.request

import json

from astropy.io import fits
from astropy.utils.data import download_file
import astropy.units as u
from astropy import wcs
from astropy.wcs import WCS
from astropy.visualization import ImageNormalize, ManualInterval, LogStretch, LinearStretch, AsinhStretch

In [2]:
import matplotlib.pyplot as plt
import matplotlib as mpl

In [3]:
# The calwebb_spec and spec3 pipelines
from jwst.pipeline import Spec2Pipeline
from jwst.pipeline import Spec3Pipeline

# the level1 pipeline:
from jwst.pipeline import Detector1Pipeline

# individual steps - probably remove this later, running step-by-step is kind of a pain
from jwst.assign_wcs import AssignWcsStep
from jwst.assign_wcs import nirspec
from jwst.background import BackgroundStep
from jwst.imprint import ImprintStep
from jwst.msaflagopen import MSAFlagOpenStep
from jwst.extract_2d import Extract2dStep
from jwst.srctype import SourceTypeStep
from jwst.wavecorr import WavecorrStep
from jwst.flatfield import FlatFieldStep
from jwst.pathloss import PathLossStep
from jwst.photom import PhotomStep
from jwst.cube_build import CubeBuildStep
from jwst.extract_1d import Extract1dStep

# data models
from jwst import datamodels

# association file utilities
from jwst.associations import asn_from_list as afl # Tools for creating association files
from jwst.associations.lib.rules_level2_base import DMSLevel2bBase # Definition of a Lvl2 association file
from jwst.associations.lib.rules_level3_base import DMS_Level3_Base # Definition of a Lvl3 association file

In [4]:
# copied over from JWebbinars notebooks. I don't really use it here, so can probably be deleted?
def show_image(data_2d, vmin, vmax, xsize=15, ysize=15, title=None, aspect=1, scale='log', units='MJy/sr'):
    """Function to generate a 2D, log-scaled image of the data
    
    Parameters
    ----------
    data_2d : numpy.ndarray
        2D image to be displayed
        
    vmin : float
        Minimum signal value to use for scaling
        
    vmax : float
        Maximum signal value to use for scaling
        
    title : str
        String to use for the plot title
        
    scale : str
        Specify scaling of the image. Can be 'log' or 'linear'
        
    units : str
        Units of the data. Used for the annotation in the
        color bar
    """
    if scale == 'log':
        norm = ImageNormalize(data_2d, interval=ManualInterval(vmin=vmin, vmax=vmax),
                              stretch=LogStretch())
    elif scale == 'linear':
        norm = ImageNormalize(data_2d, interval=ManualInterval(vmin=vmin, vmax=vmax),
                              stretch=LinearStretch())
    elif scale == 'Asinh':
        norm = ImageNormalize(data_2d, interval=ManualInterval(vmin=vmin, vmax=vmax),
                              stretch=AsinhStretch())
    fig = plt.figure(figsize=(xsize, ysize))
    ax = fig.add_subplot(1, 1, 1)
    im = ax.imshow(data_2d, origin='lower', norm=norm, aspect=aspect, cmap='gist_earth')

    fig.colorbar(im, label=units)
    plt.xlabel('Pixel column')
    plt.ylabel('Pixel row')
    if title:
        plt.title(title)


## Stage 1 - Detector-level processing
The first time this runs it will be glacially slow, as many GB of reference files need to be downloaded.
Changing the maximum_cores variable in the parameter file can speed up this step. The default is "None", which uses one core. Other options are "quarter", "half", and "all", which will use 1/4, 1/2, or all available cores respectively.

Currently the cell below processes all uncal.fits files together, but I need to change this. Later we want the "science" and "background" exposures to be treated seperately. I moved files over to different sub-directories manually for this run, but it will be far easier to just set this up from the start. 

In [4]:
# test by TAH to see if the files are accessible
files = glob.glob(input_path + '*nrs*/*_uncal.fits') #list the uncalibrated (level 1b) files.
files = sorted(files)

ifu,sky = 0,0

for exposure in files:
    test = fits.open(exposure)
    head = test[0].header
    if head['TARGPROP'] == 'SGAS1723-IFU': ifu+=1
    elif head['TARGPROP'] == 'SGAS1723-SKY': sky+=1
    else: print('not target')
    
print(f'Number of IFU: {ifu}, Number of sky: {sky}')

Number of IFU: 64, Number of sky: 55


In [5]:
# run first step of the pipeline - should be same for nircam and nirspec
# copied from Jane/Jared's notebook on nircam reductions
files = glob.glob(input_path + '*nrs*/*_uncal.fits') #list the uncalibrated (level 1b) files.
files = sorted(files)
for exposure in files:
    Detector1Pipeline.call(exposure,#config_file=detector1_parameter_file, maximum_cores=5,\
                           output_dir=output_path + 'L2a/',\
                           output_file=f'{os.path.basename(exposure)[:-11]}')

2022-09-28 18:22:28,854 - stpipe - WARNING - /Users/tahutch1/programs/anaconda3/envs/jwst-test/lib/python3.10/site-packages/asdf/entry_points.py:46: AsdfWarning: asdf.extensions plugin from package asdf-astropy==0.2.2 failed to load:

ContextualVersionConflict: (jsonschema 4.16.0 (/Users/tahutch1/programs/anaconda3/envs/jwst-test/lib/python3.10/site-packages), Requirement.parse('jsonschema<4.10.0,>=4.0.1'), {'asdf'})
  warnings.warn(

2022-09-28 18:22:28,873 - stpipe - WARNING - /Users/tahutch1/programs/anaconda3/envs/jwst-test/lib/python3.10/site-packages/asdf/entry_points.py:46: AsdfWarning: asdf.extensions plugin from package gwcs==0.18.2 failed to load:

ContextualVersionConflict: (jsonschema 4.16.0 (/Users/tahutch1/programs/anaconda3/envs/jwst-test/lib/python3.10/site-packages), Requirement.parse('jsonschema<4.10.0,>=4.0.1'), {'asdf'})
  warnings.warn(

2022-09-28 18:22:29,007 - stpipe - WARNING - /Users/tahutch1/programs/anaconda3/envs/jwst-test/lib/python3.10/site-packages/asdf/

FileNotFoundError: Unable to fetch schema from non-file URL: http://stsci.edu/schemas/jwst_datamodel/level1b.schema

The above block works to run the pipeline with default parameters. To control parameters directly from the notebook, the below version (should) work. Simply add "det1.[step].[param] = [val]" before calling "det1(exposure)"
For example, to utilize multiple cores, add the line "det1.jump.maximum_cores = 'half'" and "det1.ramp_fit.maximum_cores = 'half'" to use half the available cores in those two steps.

In [ ]:
# different version running the Stage 1 pipeline w/ more (easier) control
# using the run method allows more flexibility than the call method for some odd reason
files_27 = glob.glob(input_path + 'jw01355027*/*_uncal.fits') #list the uncalibrated (level 1b) files.
files_27 = sorted(files_27)
for exposure in files_27:
    det1 = Detector1Pipeline()
    det1.output_dir = output_path + 'L2a/obs27/'
    det1.save_results = True
    det1.save_parameters = True
    det1.jump.maximum_cores = 'half'
    det1.ramp_fit.maximum_cores = 'half'
    det1(exposure)

In [ ]:
# and again for the obs28 files (which should be the background exposures)
files_28 = glob.glob(input_path + 'jw01355028*/*_uncal.fits') #list the uncalibrated (level 1b) files.
files_28 = sorted(files_28)
for exposure in files_28:
    det1 = Detector1Pipeline()
    det1.output_dir = output_path + 'L2a/obs28/'
    det1.save_results = True
    det1.save_parameters = True
    det1.jump.maximum_cores = 'half'
    det1.ramp_fit.maximum_cores = 'half'
    #det1(exposure)

## Stage 2 - Produce calibrated exposures
As mentioned above, these next two cells assume that you have two directories with the L2a outputs from the Stage 1 pipeline. Currently I'm calling these "obs27" and "obs28", as they are labelled in the uncal.fits file names. "obs27" is the science target, and "obs28" is the background target. 

I split this into two calls, one for the science files (first cell) and one for the background files (second cell). I'm processing them the same way so far, using default pipeline parameters, but that could change in the future. 

As the pipeline is set up here, no background subtraction is performed in this stage. This version is building toward a "master" background subtraction step in the stage 3 pipeline, which will use the "x1d.fits" files from the background observations to define a background. 

In [ ]:
# run the second step of the pipeline
# So far this will just process each exposure individually

files = glob.glob(output_path + 'L2a/obs27/' + '*_rate.fits') #list the rate files.
files = sorted(files)
for exposure in files:              
    Spec2Pipeline.call(exposure,#config_file=image2_parameter_file,\
                        output_dir=output_path + 'L2b/obs27/', \
                        output_file=f'{os.path.basename(exposure)[:-10]}')

In [ ]:
# run L2 pipeline for background observations (28 in current notation)
files = glob.glob(output_path + 'L2a/obs28/' + '*_rate.fits') #list the rate files.
files = sorted(files)
for exposure in files:              
    Spec2Pipeline.call(exposure,#config_file=image2_parameter_file,\
                        output_dir=output_path + 'L2b/obs28/', \
                        output_file=f'{os.path.basename(exposure)[:-10]}')

As with Stage 1, I'm adding more code blocks here to call the pipeline a different, more flexible way. 

In [ ]:
files_27 = glob.glob(output_path + 'L2a/obs27/' + '*_rate.fits') #list the rate files.
files_27 = sorted(files_27)
for exposure in files_27:              
    spec2 = Spec2Pipeline()
    spec2.output_dir = output_path + 'L2b/obs27/'
    spec2.save_parameters = True
    spec2.save_results = True
    spec2(exposure)

In [ ]:
files_28 = glob.glob(output_path + 'L2a/obs28/' + '*_rate.fits') #list the rate files.
files_28 = sorted(files_28)
for exposure in files_28:              
    spec2 = Spec2Pipeline()
    spec2.output_dir = output_path + 'L2b/obs28/'
    spec2.save_parameters = True
    spec2.save_results = True
    spec2(exposure)

## Stage 3 - Creating final data cubes
This stage compiles the individual calibrated exposures into a single, (theoretically) science-ready data cube. We first have to define an association file, which tells the pipeline which files to use as science exposures, and which files to use as backgrounds.

As mentioned above, for this iteration we are doing a "master" background subtraction at this stage. This creates a single 1-D spectrum from an average of the dedicated background exposures (using the extracted 1-D spectra from the previous stage). This averaged spectrum is then universally subtracted from the science data. This type of background subtraction may or may not be the right way to proceed. This is how we find out! 

In [ ]:
# BELOW COPIED FROM DAVID LAW'S MIRI MRS NOTEBOOK: 
# https://github.com/STScI-MIRI/MRS-ExampleNB/blob/main/Flight_Notebook1/MRS_FlightNB1.ipynb
# 
# Define a useful function to write out a Lvl3 association file from an input list
# Note that any background exposures have to be of type x1d.
def writel3asn(scifiles, bgfiles, asnfile, prodname):
    # Define the basic association of science files
    asn = afl.asn_from_list(scifiles, rule=DMS_Level3_Base, product_name=prodname)
        
    # Add background files to the association
    if bgfiles:
        nbg=len(bgfiles)
        for ii in range(0,nbg):
            asn['products'][0]['members'].append({'expname': bgfiles[ii], 'exptype': 'background'})
        
    # Write the association to a json file
    _, serialized = asn.dump()
    with open(asnfile, 'w') as outfile:
        outfile.write(serialized)

In [ ]:
# Make association files - use obs27 for data, obs28 for background
calfiles = glob.glob(output_path + 'L2b/obs27/*cal.fits')
#calfiles = glob.glob(output_path + 'L2b/obs28/*cal.fits') # testing w/ background images

bkgfiles = glob.glob(output_path + 'L2b/obs28/*x1d.fits')

asnfile = os.path.join(output_path, 'L3/L3asn.json')
#asnfile = os.path.join(output_path, 'L3/bkg/L3bg_asn.json') # testing w/ bkg images

writel3asn(calfiles, bkgfiles, asnfile, 'Level3')
#writel3asn(calfiles, None, asnfile, 'Level3BG') # testing w/ bkg images

In [ ]:
# Next step, call stage 3 pipeline on new asn file
asn = os.path.join(output_path, 'L3/L3_asn.json')

# setting it up this way (rather than with "call") gives a bit more flexibility to edit parameters on the fly
spec3 = Spec3Pipeline()
spec3.output_dir = output_path + 'L3/'
spec3.save_parameters = True
spec3.save_results = True # DON'T FORGET THIS OR YOU'LL WASTE SEVERAL HOURS FOR NAUGHT!

spec3.run(asn)